### Example 2 - Book Data: Compute Average Daily Spread

In this example, we're going to compute the average daily spread for Tritanium at Jita IV - Moon 4 - Caldari Navy Assembly Plant in The Forge.  The average daily spread is the average of the spread computed for each of the 288 snapshots which make up a day of order book data \(5 minute snapshots, 12 per hour, 288 per day\).  As with Example 1, we'll first show how to compute spread from first principles, then we'll introduce libraries which make the process easier.


In [1]:
# Standard imports
import pandas as pd
import numpy as np
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
import datetime
%matplotlib inline
# EveKit imports
from evekit.reference import Client
from evekit.util import convert_raw_time

In [2]:
# Settings for the day we want to compute
# We retreive type, region and station ID from the SDE
sde_client = Client.SDE.get()
type_query = "{values: ['Tritanium']}"
region_query = "{values: ['The Forge']}"
station_query = "{values: ['Jita IV - Moon 4 - Caldari Navy Assembly Plant']}"
type_id = sde_client.Inventory.getTypes(typeName=type_query).result()[0][0]['typeID']
region_id = sde_client.Map.getRegions(regionName=region_query).result()[0][0]['regionID']
station_id = sde_client.Station.getStations(stationName=station_query).result()[0][0]['stationID']
compute_date = convert_raw_time(1483228800000) # 2017-01-01 12:00 AM UTC
print("Using type_id=%d, region_id=%d, station_id=%d at %s" % (type_id, region_id, station_id, str(compute_date)))

Using type_id=34, region_id=10000002, station_id=60003760 at 2017-01-01 00:00:00


In [3]:
# Let's develop code for computing spread from an order book retrieved from the Orbital Enterprises market data service
# 
# To start, we'll need the client for the service.  We'll also fetch the first book available on our compute date.
mdc_client = Client.MarketData.get()
sample_book = mdc_client.MarketData.book(typeID=type_id, regionID=region_id, date=str(compute_date) + " UTC").result()[0]
sample_book


{'bookTime': 1483228800000,
 'orders': [{'buy': True,
   'duration': 90,
   'issued': 1481705362000,
   'locationID': 60002242,
   'minVolume': 1,
   'orderID': 4708935394,
   'orderRange': 'solarsystem',
   'price': 5,
   'regionID': 10000002,
   'typeID': 34,
   'volume': 43758803,
   'volumeEntered': 200000000},
  {'buy': True,
   'duration': 90,
   'issued': 1483056665000,
   'locationID': 60001591,
   'minVolume': 1,
   'orderID': 4719143619,
   'orderRange': 'station',
   'price': 4.86,
   'regionID': 10000002,
   'typeID': 34,
   'volume': 13303703,
   'volumeEntered': 100000000},
  {'buy': True,
   'duration': 30,
   'issued': 1483014155000,
   'locationID': 60001594,
   'minVolume': 1,
   'orderID': 4731076447,
   'orderRange': 'station',
   'price': 4.85,
   'regionID': 10000002,
   'typeID': 34,
   'volume': 13381581,
   'volumeEntered': 20000000},
  {'buy': True,
   'duration': 90,
   'issued': 1483020345000,
   'locationID': 60002818,
   'minVolume': 1,
   'orderID': 47311

In [4]:
# Orders are ordered with buys first, descending in price, followed by sells in ascending price
# Let's pull these out to simplify things
buy = [x for x in sample_book['orders'] if x['buy'] and x['locationID'] == station_id ]
sell = [x for x in sample_book['orders'] if not x['buy'] and x['locationID'] == station_id ]

In [5]:
# By construction, spread is now just the difference in price between the first buy order and 
# the first sell order.  However, there are some corner cases where one or both of these
# lists are empty.  In those cases, there isn't a spread.  Here's a function which 
# computes the spread
def compute_spread(buy, sell):
    if len(buy) == 0 or len(sell) == 0:
        return None
    return sell[0]['price'] - buy[0]['price']

compute_spread(buy, sell)


0.3200000000000003

In [6]:
# All that remains is to compute the spread for all snapshots for the target day.
# We'll do that by iterating over 288 intervals from our start time, retrieving 
# the order book at each step.  This will require 288 calls to the market data service
# so this will take a few minutes to run.
current_time = compute_date
five_minute_delta = datetime.timedelta(minutes=5)
spreads = []
for _ in range(288):
    print("Computing spread for %s" % str(current_time), end="...")
    next_book = mdc_client.MarketData.book(typeID=type_id, regionID=region_id, date=str(current_time) + " UTC").result()[0]
    buy = [x for x in next_book['orders'] if x['buy'] and x['locationID'] == station_id ]
    sell = [x for x in next_book['orders'] if not x['buy'] and x['locationID'] == station_id ]
    next_spread = compute_spread(buy, sell)
    if next_spread is not None:
        spreads.append(next_spread)
    current_time += five_minute_delta
    print("done")

np.average(spreads)

Computing spread for 2017-01-01 00:00:00...done
Computing spread for 2017-01-01 00:05:00...done
Computing spread for 2017-01-01 00:10:00...done
Computing spread for 2017-01-01 00:15:00...done
Computing spread for 2017-01-01 00:20:00...done
Computing spread for 2017-01-01 00:25:00...done
Computing spread for 2017-01-01 00:30:00...done
Computing spread for 2017-01-01 00:35:00...done
Computing spread for 2017-01-01 00:40:00...done
Computing spread for 2017-01-01 00:45:00...done
Computing spread for 2017-01-01 00:50:00...done
Computing spread for 2017-01-01 00:55:00...done
Computing spread for 2017-01-01 01:00:00...done
Computing spread for 2017-01-01 01:05:00...done
Computing spread for 2017-01-01 01:10:00...done
Computing spread for 2017-01-01 01:15:00...done
Computing spread for 2017-01-01 01:20:00...done
Computing spread for 2017-01-01 01:25:00...done
Computing spread for 2017-01-01 01:30:00...done
Computing spread for 2017-01-01 01:35:00...done
Computing spread for 2017-01-01 01:40:00

0.24631944444444442

In [7]:
# Now let's look at library support which eliminates or simplifies some of the steps above.
#
# As in the first example, we start with library functions that can be used to download
# book data to local storage.  Let's say it again so we don't forget: book data is quite large,
# make sure you're on a reasonably fast connection before you download lots of data
from evekit.online.Download import download_order_book_range

# The book downloader lets you filter to specific types and regions for a date range.
# In this example, we only need Tritanium in The Forge on our target date, so this should
# download relatively quickly.  We'll store the download in our local directory in "tree"
# format (i.e. YYYY/MM/DD/files...)
download_order_book_range([compute_date], ".", types=[type_id], regions=[region_id], config={'verbose': True, 'tree': True})


In [8]:
# Previously, we iterated through each snapshot for our target date.  We can do this 
# a bit more tersely using the OrderBook library and loading the data in a Pandas DataFrame.
# This call creates a DataFrame where each row is an individual order indexed by 
# the time of the snapshot containing the order.  The DataFrame also contains type_id
# and region_id columns which can be used to filter as needed.
from evekit.marketdata import OrderBook

order_book = OrderBook.get_data_frame(dates=[compute_date], types=[type_id], regions=[region_id], 
                                      config=dict(local_storage=".", tree=True, skip_missing=True, verbose=True))

order_book

Retrieving 2017-01-01 00:00:00...done


,buy,date,duration,issued,location_id,min_volume,order_id,order_range,price,region_id,type_id,volume,volume_entered
2017-01-01 00:00:00,True,2017-01-01 00:00:00,90,2016-12-14 08:49:22,60002242,1,4708935394,solarsystem,5.00,10000002,34,43758803,200000000
2017-01-01 00:00:00,True,2017-01-01 00:00:00,90,2016-12-30 00:11:05,60001591,1,4719143619,station,4.86,10000002,34,13303703,100000000
2017-01-01 00:00:00,True,2017-01-01 00:00:00,30,2016-12-29 12:22:35,60001594,1,4731076447,station,4.85,10000002,34,13381581,20000000
2017-01-01 00:00:00,True,2017-01-01 00:00:00,90,2016-12-29 14:05:45,60002818,1,4731159119,station,4.83,10000002,34,9718405,30000000
2017-01-01 00:00:00,True,2017-01-01 00:00:00,90,2016-12-29 15:06:24,60010819,1,4731212373,station,4.81,10000002,34,996935,1000000
2017-01-01 00:00:00,True,2017-01-01 00:00:00,90,2016-12-31 17:20:36,60004051,1,4733726383,3,4.75,10000002,34,4900182,5000000
2017-01-01 00:00:00,True,2017-01-01 00:00:00,90,2016-12-29 17:47:28,60003211,1000,4731376068,2,4.70,10000002,34,776638,5000000
2017-01-01 00:00:00,True,2017-01-01 00:00:00,30,2016-12-13 03:24:26,60003055,1,4711604426,station,4.67,10000002,34,5000000,5000000
2017-01-01 00:00:00,True,2017-01-01 00:00:00,90,2016-12-31 18:00:40,60003760,1,4733760768,1,4.66,10000002,34,1702478781,2000000000
2017-01-01 00:00:00,True,2017-01-01 00:00:00,30,2016-12-04 18:43:39,60003760,1,4701276671,station,4.65,10000002,34,78774738,200000000


In [9]:
# DataFrame operations allow us to compute average spread by concatenating a few basic operations
# But first, let's walk through it step by step so we can explain each operation.
#
# We start by eliminating all orders that aren't at our target station
filtered = order_book[order_book.location_id == station_id]

filtered

,buy,date,duration,issued,location_id,min_volume,order_id,order_range,price,region_id,type_id,volume,volume_entered
2017-01-01 00:00:00,True,2017-01-01 00:00:00,90,2016-12-31 18:00:40,60003760,1,4733760768,1,4.66,10000002,34,1702478781,2000000000
2017-01-01 00:00:00,True,2017-01-01 00:00:00,30,2016-12-04 18:43:39,60003760,1,4701276671,station,4.65,10000002,34,78774738,200000000
2017-01-01 00:00:00,True,2017-01-01 00:00:00,90,2016-12-03 07:55:24,60003760,1,4699131616,station,4.62,10000002,34,100000000,100000000
2017-01-01 00:00:00,True,2017-01-01 00:00:00,90,2016-12-04 17:22:09,60003760,1,4701158083,station,4.60,10000002,34,100000000,100000000
2017-01-01 00:00:00,True,2017-01-01 00:00:00,90,2016-12-30 00:46:00,60003760,1,4731817819,station,4.05,10000002,34,10000000,10000000
2017-01-01 00:00:00,True,2017-01-01 00:00:00,90,2016-12-24 13:14:47,60003760,1,4725469328,station,4.01,10000002,34,100000000,100000000
2017-01-01 00:00:00,True,2017-01-01 00:00:00,90,2016-12-28 16:30:47,60003760,1,4730119998,5,3.92,10000002,34,18239,100000
2017-01-01 00:00:00,True,2017-01-01 00:00:00,30,2016-12-22 14:47:46,60003760,1,4723166699,station,3.54,10000002,34,1000000,1000000
2017-01-01 00:00:00,True,2017-01-01 00:00:00,90,2016-12-30 21:49:51,60003760,1,4732829596,station,3.50,10000002,34,432000,432000
2017-01-01 00:00:00,True,2017-01-01 00:00:00,90,2016-11-30 22:12:33,60003760,1,4696407011,station,2.03,10000002,34,6500000,6500000


In [10]:
# Since spread is computed by snapshot, we need to recover the snapshots which we can
# do by grouping by index (recall that the index is essentially the snapshot time of
# each order)
#
groups = filtered.groupby(filtered.index)

groups

In [11]:
# Each group in the group object is a DataFrame representing a snapshot indexed by snapshot time
#
df = groups.get_group(filtered.index[0])

df

,buy,date,duration,issued,location_id,min_volume,order_id,order_range,price,region_id,type_id,volume,volume_entered
2017-01-01,True,2017-01-01,90,2016-12-31 18:00:40,60003760,1,4733760768,1,4.66,10000002,34,1702478781,2000000000
2017-01-01,True,2017-01-01,30,2016-12-04 18:43:39,60003760,1,4701276671,station,4.65,10000002,34,78774738,200000000
2017-01-01,True,2017-01-01,90,2016-12-03 07:55:24,60003760,1,4699131616,station,4.62,10000002,34,100000000,100000000
2017-01-01,True,2017-01-01,90,2016-12-04 17:22:09,60003760,1,4701158083,station,4.60,10000002,34,100000000,100000000
2017-01-01,True,2017-01-01,90,2016-12-30 00:46:00,60003760,1,4731817819,station,4.05,10000002,34,10000000,10000000
2017-01-01,True,2017-01-01,90,2016-12-24 13:14:47,60003760,1,4725469328,station,4.01,10000002,34,100000000,100000000
2017-01-01,True,2017-01-01,90,2016-12-28 16:30:47,60003760,1,4730119998,5,3.92,10000002,34,18239,100000
2017-01-01,True,2017-01-01,30,2016-12-22 14:47:46,60003760,1,4723166699,station,3.54,10000002,34,1000000,1000000
2017-01-01,True,2017-01-01,90,2016-12-30 21:49:51,60003760,1,4732829596,station,3.50,10000002,34,432000,432000
2017-01-01,True,2017-01-01,90,2016-11-30 22:12:33,60003760,1,4696407011,station,2.03,10000002,34,6500000,6500000


In [12]:
# We can implement a simple function to compute the spread given a group object DataFrame.
# Note that we still have to handle the case where there may be no buys or sells.  When
# we encounter such cases, we return a NaN which is conveniently ignored by the Pandas
# mean function.
#
def spread_df(df):
    if df[df.buy == True].index.size == 0 or df[df.buy == False].index.size == 0:
        return np.NaN

    return df[df.buy == False].sort_values('price', ascending=True).ix[0].price - \
           df[df.buy == True].sort_values('price', ascending=False).ix[0].price

spread_df(df)


0.32000000000000028

In [13]:
# Finally, we can apply our spread function to the groups and compute the mean (which excludes NaN automatically)
groups.apply(spread_df).mean()


0.24631944444444442

In [14]:
# Here's the whole thing as a one liner (except for the spread_df function)
#
order_book[order_book.location_id == station_id].groupby(order_book[order_book.location_id == station_id].index)\
                                                .apply(spread_df)\
                                                .mean()

0.24631944444444442